In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

### Loading Dataset

In [3]:
from datasets import load_dataset
dataDir='dataset/IK_NLP_22_PESTYLE'
dataset = load_dataset("GroNLP/ik-nlp-22_pestyle", "full", data_dir=dataDir)

Using custom data configuration full-c8d896e19c9bb48e
Reusing dataset ik_nlp22_pe_style (C:\Users\amit\.cache\huggingface\datasets\GroNLP___ik_nlp22_pe_style\full-c8d896e19c9bb48e\1.0.0\3bbf0fda4806257149c2beb42c6cd20db6f79dac9ae2498f44be55fa7a953d51)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset.shape

{'train': (1170, 30)}

In [5]:
training_whole_dataset = dataset["train"]
new_df= pd.DataFrame(data=training_whole_dataset)
new_df.shape

(1170, 30)

In [6]:
# Installing all requirements 
!pip install spacy transformers sentencepiece datasets scikit-learn pandas

#We first download the "it_core_new_sm" for italian language and  "en_core_web_sm" for English 
# to use in tokenizer in order to get tokens, POS taggs 
!python -m spacy download it_core_news_sm en_core_web_sm
!python -m spacy download en_core_web_sm

2022-04-10 00:25:55.461793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-10 00:25:55.461832: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



[+] Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


2022-04-10 00:26:12.927185: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-10 00:26:12.927293: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Import spacy library

In [7]:

import spacy
#from spacy.lang.it.examples import sentences 
"""from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
"""
"""model_name = "dbmdz/bert-base-italian-cased"
tokenizer = BertTokenizer.from_pretrained(model_name) #'bert-base-uncased')"""
'bert-base-multilingual-cased'
"""from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-italian-cased')
model = BertModel.from_pretrained("dbmdz/bert-base-italian-cased")"""
"""import nltk
import string
nltk.download('punkt')"""
 
# Define the model repo
"""model_name = ''#"mrm8488/mbart-large-finetuned-opus-it-en-translation" 
identifier="Helsinki-NLP/opus-mt-it-en"

from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained(identifier)"""




'model_name = \'\'#"mrm8488/mbart-large-finetuned-opus-it-en-translation" \nidentifier="Helsinki-NLP/opus-mt-it-en"\n\nfrom transformers import BartTokenizer\ntokenizer = BartTokenizer.from_pretrained(identifier)'

### Initializing ...
### Loading Spacy and models
    We load both italian and english language models with spacy in order to use them for tokenizing and POS tagging purposes.
### POS tags extraction functions
    In this part of code, we defined some usefull function applying in the preprocessing operation to extract POS taggs from the input text src_text, tgt_text. Beacuse of mt_text masking in test_modality and test_edit datasets, we would not extract the mt_text.     


In [8]:

nlp = spacy.load('it_core_news_sm')
nlpSrc = spacy.load('en_core_web_sm')
#nlp = spacy.load('it_core_news_md')

def clean_text(text):
    doc = nlp(text)
    print_attributes(doc)
    return getCountOfPOStagging(doc)

NOUN='NOUN'
VERB='VERB'
ADJ='ADJ'
ADV='ADV'
PROPN='PROPN'
DET='DET'
PRT='PRT'
CONJ='CONJ'
PUNCT='PUNCT'
ADP='ADP'
AUX='AUX'
CCONJ='CCONJ'
INTJ='INTJ'
NUM='NUM'
PART='PART'
PRON='PRON'
SCONJ='SCONJ'
SYM='SYM'
lst=[NOUN,VERB,ADJ,ADV,PROPN,PUNCT,ADP,AUX,CONJ,CCONJ,INTJ,NUM,PART,PRON,SCONJ,SYM]

def checkWordsInVocabDictionary(text):
    count=0
    doc=nlp(text)
    for token in doc:
        tt = nlp.vocab.strings[token.text]
        if tt in nlp.vocab:
            count +=1
    return count
#for nlp(italian)
def getNumberOfTokens(text):
    count=0
    doc=nlp(text)
    for token in doc:
        count +=1
    return count
   
def getCount(text,lstPOS):
    doc = nlp(text)
    count=0
    for token in doc:
        if token.pos_ in lstPOS: count += 1 
    return count   

def countWords(text, threshold):
    doc=nlp(text)
    count=0
    for token in doc:
        if len(token.text) < threshold :
            #print(token.text +"\t"+token.pos_)
            count +=1
    return count        

# for nlp(english)
def getNumberOfTokens_Src(text):
    count=0
    doc=nlpSrc(text)
    for token in doc:
        count +=1
    return count
   
def getCount_Src(text,lstPOS):
    doc = nlpSrc(text)
    count=0
    for token in doc:
        if token.pos_ in lstPOS: count += 1 
    return count   

# @param text : the input english text from src_text
# @Param threshold: the number of 3 and 4 used to extract word with length less than 3 and 4. 
# Later we will seperate this words from the the set od DET and PUNCT words.
# DET tag categories consists of determoners and articles 
def countWords_Src(text, threshold):
    doc=nlpSrc(text)
    count=0
    for token in doc:
        if len(token.text) < threshold :
            count +=1
    return count        

#Test:
#getCountPunc('I corpi di pace armati delle Nazioni Unite,')
#I corpi di pace armati delle Nazioni Unite,



## Preprocessing 

To have a better results, We need to clean the data and add some new feature. So we provided some functions for preprocessing.
Function PreProcess is the main functin to form the train data.
The train split contains a total of 1170 triplets (or pairs, when translation from scratch is performed) annotated with behavioral data produced during the translation. The followings are the fields and their descriptions of the train dataset:
#### Train Set
**Fields**

>**item_id**	=> The sentence identifier. The first digits of the number represent the document containing the sentence, while the last digit of the number represents the sentence position inside the document. Documents can contain from 3 to 5 semantically-related sentences each.

>**subject_id**	=> The identifier for the translator performing the translation from scratch or post-editing task. Values: t1, t2 or t3.

>**modality**	=> The modality of the translation task. Values: ht (translation from scratch), pe1 (post-editing Google Translate translations), pe2 (post-editing mBART translations).


>**src_text**	=> The original source sentence extracted from Wikinews, wikibooks or wikivoyage.

>**mt_text**	=> Missing if tasktype is ht. Otherwise, contains the automatically-translated sentence before post-editing.

>**tgt_text**	=> Final sentence produced by the translator (either via translation from scratch of sl_text or post-editing mt_text)

>**edit_time**	=> Total editing time for the translation in seconds.

>**k_total**	=> Total number of keystrokes for the translation.

>**k_letter**	=> Total number of letter keystrokes for the translation.

>**k_digit**	=> Total number of digit keystrokes for the translation.

>**k_white**	=> Total number of whitespace keystrokes for the translation.

>**k_symbol**	=> Total number of symbol (punctuation, etc.) keystrokes for the translation.

>**k_nav**	=> Total number of navigation keystrokes (left-right arrows, mouse clicks) for the translation.

>**k_erase**	=> Total number of erase keystrokes (backspace, cancel) for the translation.

>**k_copy**	=> Total number of copy (Ctrl + C) actions during the translation.

>**k_cut**	=> Total number of cut (Ctrl + X) actions during the translation.

>**k_paste**	=> Total number of paste (Ctrl + V) actions during the translation.

>**n_pause_geq_300**	=> Number of pauses of 300ms or more during the translation.

>**len_pause_geq_300**	=> Total duration of pauses of 300ms or more, in milliseconds.

>**n_pause_geq_1000**	=> Number of pauses of 1s or more during the translation.

>**len_pause_geq_1000**	=> Total duration of pauses of 1000ms or more, in milliseconds.

>**num_annotations**	=> Number of times the translator focused the texbox for performing the translation of the sentence during the translation session. E.g. 1 means the translation was performed once and never revised.

>**n_insert**	=> Number of post-editing insertions (empty for modality ht) computed using the tercom library.

>**n_delete**	=> Number of post-editing deletions (empty for modality ht) computed using the tercom library.

>**n_substitute**    => Number of post-editing substitutions (empty for modality ht) computed using the tercom library.

>**n_shift**	=> Number of post-editing shifts (empty for modality ht) computed using the tercom library.

>**bleu**	=> Sentence-level BLEU score between MT and post-edited fields (empty for modality ht) computed using the SacreBLEU library with default parameters.

>**chrf**	=> Sentence-level chrF score between MT and post-edited fields (empty for modality ht) computed using the SacreBLEU library with default parameters.

>**ter**	=> Sentence-level TER score between MT and post-edited fields (empty for modality ht) computed using the tercom library.

>**aligned_edit**	=> Aligned visual representation of REF (mt_text), HYP (tl_text) and edit operations (I = Insertion, D = Deletion, S = Substitution) performed on the field. Replace \\n with \n to show the three aligned rows.


After cleaning the data, we convert the categirial labels to onhot codes with OneHot Eecoder for both Subject and Modality. Then, along with the existing features, we formed some **new features** on input texts(src_text,mt_text and tgt_text) as follows:

>**Number of words** in source,target and mt texts seperately,

>**Number of unique words** in source,target and mt texts seperately,

>**Number of unique words- translator wise** in source,target and mt texts seperately aggregated by translators,

>**Number of unique words- modality wise** in source,target and mt texts seperately aggregated by modalities,

>**Time of each word** in the entire edit time,

>**Ratio of number of 300ms and 1000ms pauses** during the edit time,

>**Number of Insert(I), Number of Delete(D) and Sunstitute(S) operations plus sum of them as IDS** features based on the Aligned_text, 

>**Linguistic features** extracted from source text and target text (number of Noun,Verb,Det,Adj,Adp,Adv,Pron,PropN,Aux,Sym,Punct,Part),

>**Number the word with length less then 3 ,4** as **AbnormalWords** which are subtracted by DETerminers(determiners,articles) and PUCTuations and Adpositions, 

>**Abnormal tokens ratio** for source and target texts based on the total number of tokens.

>**Differences of linguistic features** from source abd target text
 

#### Test Sets    
The three test splits (one per configuration) contain the same 120 entries each, following the same structure as train. Each test split omit some of the fields to prevent leakage of information.

Based on the above preprocessing function that is specialized to train set, the other preprocessing functions provided for test sets are as follows. 

>**Preprocessing_MaskSubject()**: this function adds/removes some features based on the **test_mask_subject** test data in which the **subject_id** is absent, for the main task of post-editor stylometry. 

>**Preprocessing_MaskModality()**: this function adds/removes some features based on the **test_mask_modality** test data in which the following fields are absent for the modality prediction extra task: **modality, mt_text, n_insert, n_delete, n_substitute, n_shift, ter, bleu, chrf, aligned_edit**.

>**Preprocessing_MaskTime()**: this function adds/removes some features based on the **test_mask_time** test data in which the following fields are absent for the time and pause prediction extra task: **edit_time, n_pause_geq_300, len_pause_geq_300, n_pause_geq_1000, and len_pause_geq_1000**.

<font color='red'>Note: for each preprocessing function, a corresponding createDataFrame function is provided to collect the data from preprocessing and create the final data frame that is ready to be injected into the ML model.</font>

In [9]:
# Defining constant variables
class constants:
    Param_n_pause_geq_300='n_pause_geq_300'
    Param_len_pause_geq_300='len_pause_geq_300'
    Param_n_pause_geq_1000='n_pause_geq_1000'
    Param_len_pause_geq_1000='len_pause_geq_1000'
    Param_edit_time='edit_time'
    Param_n300_ratio= 'n300_ratio'
    Param_n1000_ratio='n1000_ratio'
    Param_TER = 'ter'
    Param_BLEU='bleu'
    Param_CHRF='chrf'
    Param_num_word_src='num_word_src'
    Param_num_word_mt='num_word_mt'
    Param_num_word_tgt='num_word_tgt'
    Param_num_word_unique_mt='num_word_unique_mt'
    Param_num_word_unique_tgt='num_word_unique_tgt'
    Param_num_unique_mt_t_wise='num_unique_mt_t_wise'
    Param_num_unique_mt_m_wise='num_unique_mt_m_wise'
    Param_num_unique_tgt_t_wise='num_unique_tgt_t_wise'
    Param_num_unique_tgt_m_wise='num_unique_tgt_m_wise'
    subjects=['t1','t2','t3']
    modalities=['ht','pe1','pe2']
    s_mt_t_wise=[]
    s_tgt_t_wise=[]
    s_mt_m_wise=[]
    s_tgt_m_wise=[]
    Param_item_id='item_id'
    Param_subject_id='subject_id'
    Param_modality='modality'
    Param_src_text='src_text'
    Param_mt_text='mt_text'
    Param_tgt_text='tgt_text'
    Param_edit_time='edit_time'
    Param_k_total='k_total'
    Param_k_letter='k_letter'
    Param_k_digit='k_digit'
    Param_k_white='k_white'
    Param_k_symbol='k_symbol'
    Param_k_nav='k_nav'
    Param_k_erase='k_erase'
    Param_k_copy='k_copy'
    Param_k_cut='k_cut'
    Param_k_paste='k_paste'
    Param_num_annotations='num_annotations'
    Param_n_insert='n_insert'
    Param_n_delete='n_delete'
    Param_n_substitute='n_substitute'
    Param_n_shift='n_shift'
    Param_aligned_edit='aligned_edit'
    Param_seperated_aligned_edit='seperated_aligned_edit'
    Param_num_of_I='num_of_I'
    Param_num_of_D='num_of_D'
    Param_num_of_S='num_of_S'
    Param_num_of_IDS='num_of_IDS'
    Param_translator1='t1'
    Param_translator2='t2'
    Param_translator3='t3'
    Param_modality_ht='ht'
    Param_modality_pe1='pe1'
    Param_modality_pe2='pe2'
    Param_num_words_per_edit_time='word_time'
    #tgt linguistic featurs name
    Param_tgt_countWord4='tgt_countWord4'
    Param_tgt_countWord3='tgt_countWord3'
    Param_tgt_totalNumberOfTokens='tgt_totalNumberOfTokens'
    Param_tgt_ratioAbnormal='tgt_ratioAbnormal'
    Param_tgt_countAbnormlWord='tgt_countAbnormlWord'
    Param_tgt_NOUN='tgt_NOUN'
    Param_tgt_VERB='tgt_VERB'
    Param_tgt_ADJ='tgt_ADJ'
    Param_tgt_ADV='tgt_ADV'
    Param_tgt_PROPN='tgt_PROPN'
    Param_tgt_DET='tgt_DET'
    Param_tgt_PRT='tgt_PRT'
    Param_tgt_CONJ='tgt_CONJ'
    Param_tgt_PUNCT='tgt_PUNCT'
    Param_tgt_ADP='tgt_ADP'
    Param_tgt_AUX='tgt_AUX'
    Param_tgt_CCONJ='tgt_CCONJ'
    Param_tgt_INTJ='tgt_INTJ'
    Param_tgt_NUM='tgt_NUM'
    Param_tgt_PART='tgt_PART'
    Param_tgt_PRON='tgt_PRON'
    Param_tgt_SCONJ='tgt_SCONJ'
    Param_tgt_SYM='tgt_SYM'

    #src linguistic featurs name
    Param_src_countWord4='src_countWord4'
    Param_src_countWord3='src_countWord3'
    Param_src_totalNumberOfTokens='src_totalNumberOfTokens'
    Param_src_ratioAbnormal='src_ratioAbnormal'
    Param_src_countAbnormlWord='src_countAbnormlWord'
    Param_src_NOUN='src_NOUN'
    Param_src_VERB='src_VERB'
    Param_src_ADJ='src_ADJ'
    Param_src_ADV='src_ADV'
    Param_src_PROPN='src_PROPN'
    Param_src_DET='src_DET'
    Param_src_PRT='src_PRT'
    Param_src_CONJ='src_CONJ'
    Param_src_PUNCT='src_PUNCT'
    Param_src_ADP='src_ADP'
    Param_src_AUX='src_AUX'
    Param_src_CCONJ='src_CCONJ'
    Param_src_INTJ='src_INTJ'
    Param_src_NUM='src_NUM'
    Param_src_PART='src_PART'
    Param_src_PRON='src_PRON'
    Param_src_SCONJ='src_SCONJ'
    Param_src_SYM='src_SYM'

    #linguistic featurs diff
    Param_diff_countWord4='diff_countWord4'
    Param_diff_countWord3='diff_countWord3'
    Param_diff_totalNumberOfTokens='diff_totalNumberOfTokens'
    Param_diff_ratioAbnormal='diff_ratioAbnormal'
    Param_diff_countAbnormlWord='diff_countAbnormlWord'
    Param_diff_NOUN='diff_NOUN'
    Param_diff_VERB='diff_VERB'
    Param_diff_ADJ='diff_ADJ'
    Param_diff_ADV='diff_ADV'
    Param_diff_PROPN='diff_PROPN'
    Param_diff_DET='diff_DET'
    Param_diff_PRT='diff_PRT'
    Param_diff_CONJ='diff_CONJ'
    Param_diff_PUNCT='diff_PUNCT'
    Param_diff_ADP='diff_ADP'
    Param_diff_AUX='diff_AUX'
    Param_diff_CCONJ='diff_CCONJ'
    Param_diff_INTJ='diff_INTJ'
    Param_diff_NUM='diff_NUM'
    Param_diff_PART='diff_PART'
    Param_diff_PRON='diff_PRON'
    Param_diff_SCONJ='diff_SCONJ'
    Param_diff_SYM='diff_SYM'
    
    
# Doing preprocessing for the  train dataset
def PreProcess(df):
    
    print('Starting preprocessing ....')
    if df.isnull().sum().all().sum()>0:
        print('\n Data has some null values')
        print('\n Fill NaN value to 0')
        df=df.fillna(0)
        print('\n done')
        
    print('\n Create dummies with OneHot encoder for subjects and modalities')
    translator='subject_id'
    modality='modality'
    
    traslator_vectors = {}
    modality_vectors = {}
    i=1
    modality_vectors = pd.get_dummies(df[modality])
    traslator_vectors = pd.get_dummies(df[translator])

    for subject in constants.subjects:
        df[subject]=traslator_vectors[subject]

    for modality in constants.modalities:
        df[modality]=modality_vectors[modality]

    print('done \n Replace nan string with empty in aligned_edit and mt_text')
    
    #replace nan string in aligned_edit and mt_text
    df[constants.Param_aligned_edit].replace('nan','',inplace=True)
    df[constants.Param_mt_text].replace('nan','',inplace=True)
    
    print('done \n Creating new columns number of words and number of uniaue words for src_text, mt_text and tgt_text')
    # Number of words in src_text,mt_text,tgt_text
    df[constants.Param_num_word_src] = df['src_text'].apply(lambda x: len(str(x).split()))
    df[constants.Param_num_word_mt] = df[constants.Param_mt_text].apply(lambda x: len(str(x).split()))
    df[constants.Param_num_word_tgt] = df[constants.Param_tgt_text].apply(lambda x: len(str(x).split()))

    # time to each word in the entire edit time
    df[constants.Param_num_words_per_edit_time]=df[constants.Param_edit_time]/df[constants.Param_num_word_tgt]

    # Number of unigue words in src_text,mt_text,tgt_text
    #df["num_unique_src"] = df["src_text"].apply(lambda x: len(set(str(x).split())))
    df[constants.Param_num_word_unique_mt] = df[constants.Param_mt_text].apply(lambda x: len(set(str(x).split())))
    df[constants.Param_num_word_unique_tgt] = df[constants.Param_tgt_text].apply(lambda x: len(set(str(x).split())))

    
    # Number of unigue words in mt_text group by subject 
    constants.s_mt_t_wise=df.groupby(constants.Param_subject_id)[constants.Param_mt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_mt_t_wise = constants.s_mt_t_wise.apply(lambda x: len(set(str(x).split())))

    # Number of unigue words in tgt_text group by subject 
    constants.s_tgt_t_wise=df.groupby(constants.Param_subject_id)[constants.Param_tgt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_tgt_t_wise = constants.s_tgt_t_wise.apply(lambda x: len(set(str(x).split())))

    # Number of unigue words in mt_text group by modality 
    constants.s_mt_m_wise=df.groupby(constants.Param_modality)[constants.Param_mt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_mt_m_wise = constants.s_mt_m_wise.apply(lambda x: len(set(str(x).split())))
    
    # Number of unigue words in tgt_text group by modality
    constants.s_tgt_m_wise=df.groupby(constants.Param_modality)[constants.Param_tgt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_tgt_m_wise= constants.s_tgt_m_wise.apply(lambda x: len(set(str(x).split())))

    # Calculating Ratio 300 & 1000
    df[constants.Param_n300_ratio]=(df[constants.Param_n_pause_geq_300]/df[constants.Param_edit_time])*100
    df[constants.Param_n1000_ratio]=(df[constants.Param_n_pause_geq_1000]/df[constants.Param_edit_time])*100

    # Setting unique numbers(above values) in the dataframe
    for subject in constants.subjects:
        df.loc[(df[constants.Param_subject_id] == subject), 'num_unique_mt_t_wise']=constants.s_mt_t_wise[subject]
        df.loc[(df[constants.Param_subject_id] == subject), 'num_unique_tgt_t_wise']=constants.s_tgt_t_wise[subject]

    for modality in constants.modalities:
        df.loc[(df[constants.Param_modality] == modality), 'num_unique_mt_m_wise']=constants.s_mt_m_wise[modality]
        df.loc[(df[constants.Param_modality] == modality), 'num_unique_tgt_m_wise']=constants.s_tgt_m_wise[modality]
    
    # Aligned text extraction
    new= df[constants.Param_aligned_edit].str.split("EVAL:", n = 1, expand = True)
    df[constants.Param_seperated_aligned_edit] =new[1]
    df[constants.Param_seperated_aligned_edit].replace('NONE','',inplace=True)
    df[constants.Param_num_of_I] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('I'))-1)
    df[constants.Param_num_of_D] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('D'))-1)
    df[constants.Param_num_of_S] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('S'))-1)
    df[constants.Param_num_of_IDS] = df[constants.Param_num_of_I]+df[constants.Param_num_of_D]+df[constants.Param_num_of_S]

    
    df = df.drop(constants.Param_seperated_aligned_edit, 1)

    # linguistic extraction from src_text  + ratio Abnormal tokens
    text=constants.Param_src_text
    #getting the word with length less then 3 ,4 -> we will compare them to DETerminrs(determiners and articles) and PUCTuations later 
    # to clean it and reach to the abnormal word which machine translators might produce
    df[constants.Param_src_countWord3]=df[text].map( lambda x: countWords_Src(x,3))
    df[constants.Param_src_countWord4]=df[text].map( lambda x: countWords_Src(x,4))
    
    df[constants.Param_src_totalNumberOfTokens]=df[text].map( lambda x: getNumberOfTokens_Src(x))
    df[constants.Param_src_NOUN]=df[text].map( lambda x: getCount_Src(x,[NOUN]))
    df[constants.Param_src_VERB]=df[text].map( lambda x: getCount_Src(x,[VERB]))
    df[constants.Param_src_ADJ]=df[text].map( lambda x: getCount_Src(x,[ADJ]))
    df[constants.Param_src_ADV]=df[text].map( lambda x: getCount_Src(x,[ADV]))
    df[constants.Param_src_PROPN]=df[text].map( lambda x: getCount_Src(x,[PROPN]))
    df[constants.Param_src_PRON]=df[text].map( lambda x: getCount_Src(x,[PRON]))
    df[constants.Param_src_AUX]=df[text].map( lambda x: getCount_Src(x,[AUX]))
    df[constants.Param_src_CONJ]=df[text].map( lambda x: getCount_Src(x,[CONJ,CCONJ,SCONJ]))
    df[constants.Param_src_ADP]=df[text].map( lambda x: getCount_Src(x,[ADP]))
    df[constants.Param_src_DET]=df[text].map( lambda x: getCount_Src(x,[DET]))
    df[constants.Param_src_SYM]=df[text].map( lambda x: getCount_Src(x,[SYM]))
    df[constants.Param_src_PART]=df[text].map( lambda x: getCount_Src(x,[PART]))
    df[constants.Param_src_PUNCT]=df[text].map( lambda x: getCount_Src(x,[PUNCT]))
    df[constants.Param_src_countAbnormlWord]=(df[constants.Param_src_countWord4]+df[constants.Param_src_countWord3])-(df[constants.Param_src_DET]+df[constants.Param_src_PUNCT]+df[constants.Param_src_ADP])
    
    df[constants.Param_src_ratioAbnormal]=(df[constants.Param_src_countAbnormlWord]/df[constants.Param_src_totalNumberOfTokens])*100
    df[constants.Param_src_ratioAbnormal]=df[constants.Param_src_ratioAbnormal].replace(np.Infinity,0)
   
                                          


    # linguistic tgt_text features + ratio Abnormal tokens
    text=constants.Param_tgt_text
    df[constants.Param_tgt_countWord3]=df[text].map( lambda x: countWords(x,3))
    df[constants.Param_tgt_countWord4]=df[text].map( lambda x: countWords(x,4))
    df[constants.Param_tgt_totalNumberOfTokens]=df[text].map( lambda x: getNumberOfTokens(x))
    df[constants.Param_tgt_NOUN]=df[text].map( lambda x: getCount(x,[NOUN]))
    df[constants.Param_tgt_VERB]=df[text].map( lambda x: getCount(x,[VERB]))
    df[constants.Param_tgt_ADJ]=df[text].map( lambda x: getCount(x,[ADJ]))
    df[constants.Param_tgt_ADV]=df[text].map( lambda x: getCount(x,[ADV]))
    df[constants.Param_tgt_PROPN]=df[text].map( lambda x: getCount(x,[PROPN]))
    df[constants.Param_tgt_PRON]=df[text].map( lambda x: getCount(x,[PRON]))
    df[constants.Param_tgt_AUX]=df[text].map( lambda x: getCount(x,[AUX]))
    df[constants.Param_tgt_CONJ]=df[text].map( lambda x: getCount(x,[CONJ,CCONJ,SCONJ]))
    df[constants.Param_tgt_ADP]=df[text].map( lambda x: getCount(x,[ADP]))
    df[constants.Param_tgt_DET]=df[text].map( lambda x: getCount(x,[DET]))
    df[constants.Param_tgt_SYM]=df[text].map( lambda x: getCount(x,[SYM]))
    df[constants.Param_tgt_PART]=df[text].map( lambda x: getCount(x,[PART]))
    df[constants.Param_tgt_PUNCT]=df[text].map( lambda x: getCount(x,[PUNCT]))
    df[constants.Param_tgt_countAbnormlWord]=(df[constants.Param_tgt_countWord4]+df[constants.Param_tgt_countWord3])-(df[constants.Param_tgt_DET]+df[constants.Param_tgt_PUNCT]+df[constants.Param_tgt_ADP])
    df[constants.Param_tgt_ratioAbnormal]=(df[constants.Param_tgt_countAbnormlWord]/df[constants.Param_tgt_totalNumberOfTokens])*100
    df[constants.Param_tgt_ratioAbnormal]=df[constants.Param_tgt_ratioAbnormal]
    # linguistic featurs diff 
    df[constants.Param_diff_countWord3]=((df[constants.Param_tgt_countWord3]-df[constants.Param_src_countWord3])/df[constants.Param_tgt_countWord3])
    df[constants.Param_diff_countWord4]=((df[constants.Param_tgt_countWord4]-df[constants.Param_src_countWord4])/df[constants.Param_tgt_countWord4])
    df[constants.Param_diff_totalNumberOfTokens]=((df[constants.Param_tgt_totalNumberOfTokens]-df[constants.Param_src_totalNumberOfTokens])/df[constants.Param_tgt_totalNumberOfTokens])
    df[constants.Param_diff_NOUN]=((df[constants.Param_tgt_NOUN]-df[constants.Param_src_NOUN])/df[constants.Param_tgt_NOUN])
    df[constants.Param_diff_VERB]=((df[constants.Param_tgt_VERB]-df[constants.Param_src_VERB])/df[constants.Param_tgt_VERB])
    df[constants.Param_diff_ADJ]=((df[constants.Param_tgt_ADJ]-df[constants.Param_src_ADJ])/df[constants.Param_tgt_ADJ])
    df[constants.Param_diff_ADV]=((df[constants.Param_tgt_ADV]-df[constants.Param_src_ADV])/df[constants.Param_tgt_ADV])
    df[constants.Param_diff_PROPN]=((df[constants.Param_tgt_PROPN]-df[constants.Param_src_PROPN])/df[constants.Param_tgt_PROPN])
    df[constants.Param_diff_PRON]=((df[constants.Param_tgt_PRON]-df[constants.Param_src_PRON])/df[constants.Param_tgt_PRON])
    df[constants.Param_diff_AUX]=((df[constants.Param_tgt_AUX]-df[constants.Param_src_AUX])/df[constants.Param_tgt_AUX])
    df[constants.Param_diff_CONJ]=((df[constants.Param_tgt_CONJ]-df[constants.Param_src_CONJ])/df[constants.Param_tgt_CONJ])
    df[constants.Param_diff_ADP]=((df[constants.Param_tgt_ADP]-df[constants.Param_src_ADP])/df[constants.Param_tgt_ADP])
    df[constants.Param_diff_DET]=((df[constants.Param_tgt_DET]-df[constants.Param_src_DET])/df[constants.Param_tgt_DET])
    df[constants.Param_diff_SYM]=((df[constants.Param_tgt_SYM]-df[constants.Param_src_SYM])/df[constants.Param_tgt_SYM])
    df[constants.Param_diff_PART]=((df[constants.Param_tgt_PART]-df[constants.Param_src_PART])/df[constants.Param_tgt_PART]).replace(np.NINF,0)
    df[constants.Param_diff_PUNCT]=((df[constants.Param_tgt_PUNCT]-df[constants.Param_src_PUNCT])/df[constants.Param_tgt_PUNCT])
    df[constants.Param_diff_countAbnormlWord]=(( df[constants.Param_tgt_countAbnormlWord]- df[constants.Param_src_countAbnormlWord])/ df[constants.Param_tgt_countAbnormlWord]).replace(np.Infinity,0)
    df[constants.Param_diff_ratioAbnormal]=((df[constants.Param_tgt_ratioAbnormal]-df[constants.Param_src_ratioAbnormal])/df[constants.Param_tgt_ratioAbnormal])
    df=df.fillna(0)
    df=df.replace(np.Infinity,0)
    return df
   
def PreProcess_MaskSubject(df):
    
    print('Starting preprocessing ....')
    if df.isnull().sum().all().sum()>0:
        print('\n Data has some null values')
        print('\n Fill NaN value to 0')
        df=df.fillna(0)
        print('\n done')
        
    print('\n Create dummies with OneHot encoder for subjects and modalities')
    #translator='subject_id'
    modality='modality'
    
    #traslator_vectors = {}
    modality_vectors = {}

    #traslator_vectors = pd.get_dummies(df[translator])
    modality_vectors = pd.get_dummies(df[modality])
    i=1
    #for subject in constants.subjects:
        #df[subject]=traslator_vectors[subject]

    for modality in constants.modalities:
        df[modality]=modality_vectors[modality]

    print('done \n Replace nan string with empty in aligned_edit and mt_text')
    #replace nan string in aligned_edit and mt_text
    df[constants.Param_aligned_edit].replace('nan','',inplace=True)
    df[constants.Param_mt_text].replace('nan','',inplace=True)
    
    print('done \n Creating new columns number of words and number of uniaue words for src_text, mt_text and tgt_text')
    # Number of words in src_text,mt_text,tgt_text
    df[constants.Param_num_word_src] = df['src_text'].apply(lambda x: len(str(x).split()))
    df[constants.Param_num_word_mt] = df[constants.Param_mt_text].apply(lambda x: len(str(x).split()))
    df[constants.Param_num_word_tgt] = df[constants.Param_tgt_text].apply(lambda x: len(str(x).split()))

    # Number of unigue words in src_text,mt_text,tgt_text
    #df["num_unique_src"] = df["src_text"].apply(lambda x: len(set(str(x).split())))
    df[constants.Param_num_word_unique_mt] = df[constants.Param_mt_text].apply(lambda x: len(set(str(x).split())))
    df[constants.Param_num_word_unique_tgt] = df[constants.Param_tgt_text].apply(lambda x: len(set(str(x).split())))

     # time to each word in the entire edit time
    df[constants.Param_num_words_per_edit_time]=df[constants.Param_edit_time]/df[constants.Param_num_word_tgt]
    
    # Number of unigue words in mt_text group by subject 
    #constants.s_mt_t_wise=df.groupby(constants.Param_subject_id)[constants.Param_mt_text].agg(lambda x: ' '.join(set(x)))
    #constants.s_mt_t_wise = constants.s_mt_t_wise.apply(lambda x: len(set(str(x).split())))

    # Number of unigue words in tgt_text group by subject 
    #constants.s_tgt_t_wise=df.groupby(constants.Param_subject_id)[constants.Param_tgt_text].agg(lambda x: ' '.join(set(x)))
    #constants.s_tgt_t_wise = constants.s_tgt_t_wise.apply(lambda x: len(set(str(x).split())))

    # Number of unigue words in mt_text group by modality 
    constants.s_mt_m_wise=df.groupby(constants.Param_modality)[constants.Param_mt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_mt_m_wise = constants.s_mt_m_wise.apply(lambda x: len(set(str(x).split())))
    
    # Number of unigue words in tgt_text group by modality
    constants.s_tgt_m_wise=df.groupby(constants.Param_modality)[constants.Param_tgt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_tgt_m_wise= constants.s_tgt_m_wise.apply(lambda x: len(set(str(x).split())))

    # Calculating Ratio 300 & 1000
    df[constants.Param_n300_ratio]=(df[constants.Param_n_pause_geq_300]/df[constants.Param_edit_time])*100
    df[constants.Param_n1000_ratio]=(df[constants.Param_n_pause_geq_1000]/df[constants.Param_edit_time])*100

    # Setting unique numbers(above values) in the dataframe
    #for subject in constants.subjects:
        #df.loc[(df[constants.Param_subject_id] == subject), 'num_unique_mt_t_wise']=constants.s_mt_t_wise[subject]
        #df.loc[(df[constants.Param_subject_id] == subject), 'num_unique_tgt_t_wise']=constants.s_tgt_t_wise[subject]

    for modality in constants.modalities:
        df.loc[(df[constants.Param_modality] == modality), 'num_unique_mt_m_wise']=constants.s_mt_m_wise[modality]
        df.loc[(df[constants.Param_modality] == modality), 'num_unique_tgt_m_wise']=constants.s_tgt_m_wise[modality]
    
    # Aligned text extraction
    new= df[constants.Param_aligned_edit].str.split("EVAL:", n = 1, expand = True)
    df[constants.Param_seperated_aligned_edit] =new[1]
    df[constants.Param_seperated_aligned_edit].replace('NONE','',inplace=True)
    df[constants.Param_num_of_I] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('I'))-1)
    df[constants.Param_num_of_D] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('D'))-1)
    df[constants.Param_num_of_S] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('S'))-1)
    df[constants.Param_num_of_IDS] = df[constants.Param_num_of_I]+df[constants.Param_num_of_D]+df[constants.Param_num_of_S]
    df = df.drop(constants.Param_seperated_aligned_edit, 1)

    # linguistic src_text features + ratio Abnormal tokens
    text=constants.Param_src_text
    df[constants.Param_src_countWord3]=df[text].map( lambda x: countWords_Src(x,3))
    df[constants.Param_src_countWord4]=df[text].map( lambda x: countWords_Src(x,4))
    df[constants.Param_src_totalNumberOfTokens]=df[text].map( lambda x: getNumberOfTokens_Src(x))
    df[constants.Param_src_NOUN]=df[text].map( lambda x: getCount_Src(x,[NOUN]))
    df[constants.Param_src_VERB]=df[text].map( lambda x: getCount_Src(x,[VERB]))
    df[constants.Param_src_ADJ]=df[text].map( lambda x: getCount_Src(x,[ADJ]))
    df[constants.Param_src_ADV]=df[text].map( lambda x: getCount_Src(x,[ADV]))
    df[constants.Param_src_PROPN]=df[text].map( lambda x: getCount_Src(x,[PROPN]))
    df[constants.Param_src_PRON]=df[text].map( lambda x: getCount_Src(x,[PRON]))
    df[constants.Param_src_AUX]=df[text].map( lambda x: getCount_Src(x,[AUX]))
    df[constants.Param_src_CONJ]=df[text].map( lambda x: getCount_Src(x,[CONJ,CCONJ,SCONJ]))
    df[constants.Param_src_ADP]=df[text].map( lambda x: getCount_Src(x,[ADP]))
    df[constants.Param_src_DET]=df[text].map( lambda x: getCount_Src(x,[DET]))
    df[constants.Param_src_SYM]=df[text].map( lambda x: getCount_Src(x,[SYM]))
    df[constants.Param_src_PART]=df[text].map( lambda x: getCount_Src(x,[PART]))
    df[constants.Param_src_PUNCT]=df[text].map( lambda x: getCount_Src(x,[PUNCT]))
    df[constants.Param_src_countAbnormlWord]=(df[constants.Param_src_countWord4]+df[constants.Param_src_countWord3])-(df[constants.Param_src_DET]+df[constants.Param_src_PUNCT]+df[constants.Param_src_ADP])
    
    df[constants.Param_src_ratioAbnormal]=(df[constants.Param_src_countAbnormlWord]/df[constants.Param_src_totalNumberOfTokens])*100
    df[constants.Param_src_ratioAbnormal]=df[constants.Param_src_ratioAbnormal].replace(np.Infinity,0)
   
                                          


    # linguistic tgt_text features + ratio Abnormal tokens
    text=constants.Param_tgt_text
    df[constants.Param_tgt_countWord3]=df[text].map( lambda x: countWords(x,3))
    df[constants.Param_tgt_countWord4]=df[text].map( lambda x: countWords(x,4))
    df[constants.Param_tgt_totalNumberOfTokens]=df[text].map( lambda x: getNumberOfTokens(x))
    df[constants.Param_tgt_NOUN]=df[text].map( lambda x: getCount(x,[NOUN]))
    df[constants.Param_tgt_VERB]=df[text].map( lambda x: getCount(x,[VERB]))
    df[constants.Param_tgt_ADJ]=df[text].map( lambda x: getCount(x,[ADJ]))
    df[constants.Param_tgt_ADV]=df[text].map( lambda x: getCount(x,[ADV]))
    df[constants.Param_tgt_PROPN]=df[text].map( lambda x: getCount(x,[PROPN]))
    df[constants.Param_tgt_PRON]=df[text].map( lambda x: getCount(x,[PRON]))
    df[constants.Param_tgt_AUX]=df[text].map( lambda x: getCount(x,[AUX]))
    df[constants.Param_tgt_CONJ]=df[text].map( lambda x: getCount(x,[CONJ,CCONJ,SCONJ]))
    df[constants.Param_tgt_ADP]=df[text].map( lambda x: getCount(x,[ADP]))
    df[constants.Param_tgt_DET]=df[text].map( lambda x: getCount(x,[DET]))
    df[constants.Param_tgt_SYM]=df[text].map( lambda x: getCount(x,[SYM]))
    df[constants.Param_tgt_PART]=df[text].map( lambda x: getCount(x,[PART]))
    df[constants.Param_tgt_PUNCT]=df[text].map( lambda x: getCount(x,[PUNCT]))
    df[constants.Param_tgt_countAbnormlWord]=(df[constants.Param_tgt_countWord4]+df[constants.Param_tgt_countWord3])-(df[constants.Param_tgt_DET]+df[constants.Param_tgt_PUNCT]+df[constants.Param_tgt_ADP])
    df[constants.Param_tgt_ratioAbnormal]=(df[constants.Param_tgt_countAbnormlWord]/df[constants.Param_tgt_totalNumberOfTokens])*100
    df[constants.Param_tgt_ratioAbnormal]=df[constants.Param_tgt_ratioAbnormal]
    # linguistic featurs diff 
    df[constants.Param_diff_countWord3]=((df[constants.Param_tgt_countWord3]-df[constants.Param_src_countWord3])/df[constants.Param_tgt_countWord3])
    df[constants.Param_diff_countWord4]=((df[constants.Param_tgt_countWord4]-df[constants.Param_src_countWord4])/df[constants.Param_tgt_countWord4])
    df[constants.Param_diff_totalNumberOfTokens]=((df[constants.Param_tgt_totalNumberOfTokens]-df[constants.Param_src_totalNumberOfTokens])/df[constants.Param_tgt_totalNumberOfTokens])
    df[constants.Param_diff_NOUN]=((df[constants.Param_tgt_NOUN]-df[constants.Param_src_NOUN])/df[constants.Param_tgt_NOUN])
    df[constants.Param_diff_VERB]=((df[constants.Param_tgt_VERB]-df[constants.Param_src_VERB])/df[constants.Param_tgt_VERB])
    df[constants.Param_diff_ADJ]=((df[constants.Param_tgt_ADJ]-df[constants.Param_src_ADJ])/df[constants.Param_tgt_ADJ])
    df[constants.Param_diff_ADV]=((df[constants.Param_tgt_ADV]-df[constants.Param_src_ADV])/df[constants.Param_tgt_ADV])
    df[constants.Param_diff_PROPN]=((df[constants.Param_tgt_PROPN]-df[constants.Param_src_PROPN])/df[constants.Param_tgt_PROPN])
    df[constants.Param_diff_PRON]=((df[constants.Param_tgt_PRON]-df[constants.Param_src_PRON])/df[constants.Param_tgt_PRON])
    df[constants.Param_diff_AUX]=((df[constants.Param_tgt_AUX]-df[constants.Param_src_AUX])/df[constants.Param_tgt_AUX])
    df[constants.Param_diff_CONJ]=((df[constants.Param_tgt_CONJ]-df[constants.Param_src_CONJ])/df[constants.Param_tgt_CONJ])
    df[constants.Param_diff_ADP]=((df[constants.Param_tgt_ADP]-df[constants.Param_src_ADP])/df[constants.Param_tgt_ADP])
    df[constants.Param_diff_DET]=((df[constants.Param_tgt_DET]-df[constants.Param_src_DET])/df[constants.Param_tgt_DET])
    df[constants.Param_diff_SYM]=((df[constants.Param_tgt_SYM]-df[constants.Param_src_SYM])/df[constants.Param_tgt_SYM])
    df[constants.Param_diff_PART]=((df[constants.Param_tgt_PART]-df[constants.Param_src_PART])/df[constants.Param_tgt_PART]).replace(np.NINF,0)
    df[constants.Param_diff_PUNCT]=((df[constants.Param_tgt_PUNCT]-df[constants.Param_src_PUNCT])/df[constants.Param_tgt_PUNCT])
    df[constants.Param_diff_countAbnormlWord]=(( df[constants.Param_tgt_countAbnormlWord]- df[constants.Param_src_countAbnormlWord])/ df[constants.Param_tgt_countAbnormlWord]).replace(np.Infinity,0)
    df[constants.Param_diff_ratioAbnormal]=((df[constants.Param_tgt_ratioAbnormal]-df[constants.Param_src_ratioAbnormal])/df[constants.Param_tgt_ratioAbnormal])
    df=df.fillna(0)
    df=df.replace(np.Infinity,0)

    return df
def PreProcess_MaskModality(df):
    
    print('Starting preprocessing ....')
    if df.isnull().sum().all().sum()>0:
        print('\n Data has some null values')
        print('\n Fill NaN value to 0')
        df=df.fillna(0)
        print('\n done')
        
    print('\n Create dummies with OneHot encoder for subjects and modalities')
    translator='subject_id'
    modality='modality'
    
    traslator_vectors = {}
    #modality_vectors = {}

    traslator_vectors = pd.get_dummies(df[translator])
    #modality_vectors = pd.get_dummies(df[modality])
    i=1
    for subject in constants.subjects:
        df[subject]=traslator_vectors[subject]

    #for modality in constants.modalities:
        #df[modality]=modality_vectors[modality]

    print('done \n Replace nan string with empty in aligned_edit and mt_text')
    #replace nan string in aligned_edit and mt_text
    #df[constants.Param_aligned_edit].replace('nan','',inplace=True)
    #df[constants.Param_mt_text].replace('nan','',inplace=True)
    
    print('done \n Creating new columns number of words and number of uniaue words for src_text, mt_text and tgt_text')
    # Number of words in src_text,mt_text,tgt_text
    #df[constants.Param_num_word_src] = df['src_text'].apply(lambda x: len(str(x).split()))
    #df[constants.Param_num_word_mt] = df[constants.Param_mt_text].apply(lambda x: len(str(x).split()))
    df[constants.Param_num_word_tgt] = df[constants.Param_tgt_text].apply(lambda x: len(str(x).split()))

    # Number of unigue words in src_text,mt_text,tgt_text
    #df["num_unique_src"] = df["src_text"].apply(lambda x: len(set(str(x).split())))
    #df[constants.Param_num_word_unique_mt] = df[constants.Param_mt_text].apply(lambda x: len(set(str(x).split())))
    df[constants.Param_num_word_unique_tgt] = df[constants.Param_tgt_text].apply(lambda x: len(set(str(x).split())))

    # time to each word in the entire edit time
    df[constants.Param_num_words_per_edit_time]=df[constants.Param_edit_time]/df[constants.Param_num_word_tgt]
    
    # Number of unigue words in mt_text group by subject 
    #constants.s_mt_t_wise=df.groupby(constants.Param_subject_id)[constants.Param_mt_text].agg(lambda x: ' '.join(set(x)))
    #constants.s_mt_t_wise = constants.s_mt_t_wise.apply(lambda x: len(set(str(x).split())))

    # Number of unigue words in tgt_text group by subject 
    constants.s_tgt_t_wise=df.groupby(constants.Param_subject_id)[constants.Param_tgt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_tgt_t_wise = constants.s_tgt_t_wise.apply(lambda x: len(set(str(x).split())))

    # Number of unigue words in mt_text group by modality 
    #constants.s_mt_m_wise=df.groupby(constants.Param_modality)[constants.Param_mt_text].agg(lambda x: ' '.join(set(x)))
    #constants.s_mt_m_wise = constants.s_mt_m_wise.apply(lambda x: len(set(str(x).split())))
    
    # Number of unigue words in tgt_text group by modality
    #constants.s_tgt_m_wise=df.groupby(constants.Param_modality)[constants.Param_tgt_text].agg(lambda x: ' '.join(set(x)))
    #constants.s_tgt_m_wise= constants.s_tgt_m_wise.apply(lambda x: len(set(str(x).split())))

    # Calculating Ratio 300 & 1000
    df[constants.Param_n300_ratio]=(df[constants.Param_n_pause_geq_300]/df[constants.Param_edit_time])*100
    df[constants.Param_n1000_ratio]=(df[constants.Param_n_pause_geq_1000]/df[constants.Param_edit_time])*100

    # Setting unique numbers(above values) in the dataframe
    for subject in constants.subjects:
        #df.loc[(df[constants.Param_subject_id] == subject), 'num_unique_mt_t_wise']=constants.s_mt_t_wise[subject]
        df.loc[(df[constants.Param_subject_id] == subject), 'num_unique_tgt_t_wise']=constants.s_tgt_t_wise[subject]

    #for modality in constants.modalities:
        #df.loc[(df[constants.Param_modality] == modality), 'num_unique_mt_m_wise']=constants.s_mt_m_wise[modality]
        #df.loc[(df[constants.Param_modality] == modality), 'num_unique_tgt_m_wise']=constants.s_tgt_m_wise[modality]
    
    # Aligned text extraction
    """
    new= df[constants.Param_aligned_edit].str.split("EVAL:", n = 1, expand = True)
    df[constants.Param_seperated_aligned_edit] =new[1]
    df[constants.Param_seperated_aligned_edit].replace('NONE','',inplace=True)
    df[constants.Param_num_of_I] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('I'))-1)
    df[constants.Param_num_of_D] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('D'))-1)
    df[constants.Param_num_of_S] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('S'))-1)


    df = df.drop(constants.Param_seperated_aligned_edit, 1)
    """

    #new_df[[constants.Param_subject_id,constants.Param_aligned_edit,constants.Param_num_of_I,constants.Param_num_of_D,constants.Param_num_of_S,constants.Param_num_of_IDS]]
    
    # linguistic src_text features + ratio Abnormal tokens
    text=constants.Param_src_text
    df[constants.Param_src_countWord3]=df[text].map( lambda x: countWords_Src(x,3))
    df[constants.Param_src_countWord4]=df[text].map( lambda x: countWords_Src(x,4))
    df[constants.Param_src_totalNumberOfTokens]=df[text].map( lambda x: getNumberOfTokens_Src(x))
    df[constants.Param_src_NOUN]=df[text].map( lambda x: getCount_Src(x,[NOUN]))
    df[constants.Param_src_VERB]=df[text].map( lambda x: getCount_Src(x,[VERB]))
    df[constants.Param_src_ADJ]=df[text].map( lambda x: getCount_Src(x,[ADJ]))
    df[constants.Param_src_ADV]=df[text].map( lambda x: getCount_Src(x,[ADV]))
    df[constants.Param_src_PROPN]=df[text].map( lambda x: getCount_Src(x,[PROPN]))
    df[constants.Param_src_PRON]=df[text].map( lambda x: getCount_Src(x,[PRON]))
    df[constants.Param_src_AUX]=df[text].map( lambda x: getCount_Src(x,[AUX]))
    df[constants.Param_src_CONJ]=df[text].map( lambda x: getCount_Src(x,[CONJ,CCONJ,SCONJ]))
    df[constants.Param_src_ADP]=df[text].map( lambda x: getCount_Src(x,[ADP]))
    df[constants.Param_src_DET]=df[text].map( lambda x: getCount_Src(x,[DET]))
    df[constants.Param_src_SYM]=df[text].map( lambda x: getCount_Src(x,[SYM]))
    df[constants.Param_src_PART]=df[text].map( lambda x: getCount_Src(x,[PART]))
    df[constants.Param_src_PUNCT]=df[text].map( lambda x: getCount_Src(x,[PUNCT]))
    df[constants.Param_src_countAbnormlWord]=(df[constants.Param_src_countWord4]+df[constants.Param_src_countWord3])-(df[constants.Param_src_DET]+df[constants.Param_src_PUNCT]+df[constants.Param_src_ADP])
    
    df[constants.Param_src_ratioAbnormal]=(df[constants.Param_src_countAbnormlWord]/df[constants.Param_src_totalNumberOfTokens])*100
    df[constants.Param_src_ratioAbnormal]=df[constants.Param_src_ratioAbnormal].replace(np.Infinity,0)
   
                                          


    # linguistic tgt_text features + ratio Abnormal tokens
    text=constants.Param_tgt_text
    df[constants.Param_tgt_countWord3]=df[text].map( lambda x: countWords(x,3))
    df[constants.Param_tgt_countWord4]=df[text].map( lambda x: countWords(x,4))
    df[constants.Param_tgt_totalNumberOfTokens]=df[text].map( lambda x: getNumberOfTokens(x))
    df[constants.Param_tgt_NOUN]=df[text].map( lambda x: getCount(x,[NOUN]))
    df[constants.Param_tgt_VERB]=df[text].map( lambda x: getCount(x,[VERB]))
    df[constants.Param_tgt_ADJ]=df[text].map( lambda x: getCount(x,[ADJ]))
    df[constants.Param_tgt_ADV]=df[text].map( lambda x: getCount(x,[ADV]))
    df[constants.Param_tgt_PROPN]=df[text].map( lambda x: getCount(x,[PROPN]))
    df[constants.Param_tgt_PRON]=df[text].map( lambda x: getCount(x,[PRON]))
    df[constants.Param_tgt_AUX]=df[text].map( lambda x: getCount(x,[AUX]))
    df[constants.Param_tgt_CONJ]=df[text].map( lambda x: getCount(x,[CONJ,CCONJ,SCONJ]))
    df[constants.Param_tgt_ADP]=df[text].map( lambda x: getCount(x,[ADP]))
    df[constants.Param_tgt_DET]=df[text].map( lambda x: getCount(x,[DET]))
    df[constants.Param_tgt_SYM]=df[text].map( lambda x: getCount(x,[SYM]))
    df[constants.Param_tgt_PART]=df[text].map( lambda x: getCount(x,[PART]))
    df[constants.Param_tgt_PUNCT]=df[text].map( lambda x: getCount(x,[PUNCT]))
    df[constants.Param_tgt_countAbnormlWord]=(df[constants.Param_tgt_countWord4]+df[constants.Param_tgt_countWord3])-(df[constants.Param_tgt_DET]+df[constants.Param_tgt_PUNCT]+df[constants.Param_tgt_ADP])
    df[constants.Param_tgt_ratioAbnormal]=(df[constants.Param_tgt_countAbnormlWord]/df[constants.Param_tgt_totalNumberOfTokens])*100
    df[constants.Param_tgt_ratioAbnormal]=df[constants.Param_tgt_ratioAbnormal]
    # linguistic featurs diff 
    df[constants.Param_diff_countWord3]=((df[constants.Param_tgt_countWord3]-df[constants.Param_src_countWord3])/df[constants.Param_tgt_countWord3])
    df[constants.Param_diff_countWord4]=((df[constants.Param_tgt_countWord4]-df[constants.Param_src_countWord4])/df[constants.Param_tgt_countWord4])
    df[constants.Param_diff_totalNumberOfTokens]=((df[constants.Param_tgt_totalNumberOfTokens]-df[constants.Param_src_totalNumberOfTokens])/df[constants.Param_tgt_totalNumberOfTokens])
    df[constants.Param_diff_NOUN]=((df[constants.Param_tgt_NOUN]-df[constants.Param_src_NOUN])/df[constants.Param_tgt_NOUN])
    df[constants.Param_diff_VERB]=((df[constants.Param_tgt_VERB]-df[constants.Param_src_VERB])/df[constants.Param_tgt_VERB])
    df[constants.Param_diff_ADJ]=((df[constants.Param_tgt_ADJ]-df[constants.Param_src_ADJ])/df[constants.Param_tgt_ADJ])
    df[constants.Param_diff_ADV]=((df[constants.Param_tgt_ADV]-df[constants.Param_src_ADV])/df[constants.Param_tgt_ADV])
    df[constants.Param_diff_PROPN]=((df[constants.Param_tgt_PROPN]-df[constants.Param_src_PROPN])/df[constants.Param_tgt_PROPN])
    df[constants.Param_diff_PRON]=((df[constants.Param_tgt_PRON]-df[constants.Param_src_PRON])/df[constants.Param_tgt_PRON])
    df[constants.Param_diff_AUX]=((df[constants.Param_tgt_AUX]-df[constants.Param_src_AUX])/df[constants.Param_tgt_AUX])
    df[constants.Param_diff_CONJ]=((df[constants.Param_tgt_CONJ]-df[constants.Param_src_CONJ])/df[constants.Param_tgt_CONJ])
    df[constants.Param_diff_ADP]=((df[constants.Param_tgt_ADP]-df[constants.Param_src_ADP])/df[constants.Param_tgt_ADP])
    df[constants.Param_diff_DET]=((df[constants.Param_tgt_DET]-df[constants.Param_src_DET])/df[constants.Param_tgt_DET])
    df[constants.Param_diff_SYM]=((df[constants.Param_tgt_SYM]-df[constants.Param_src_SYM])/df[constants.Param_tgt_SYM])
    df[constants.Param_diff_PART]=((df[constants.Param_tgt_PART]-df[constants.Param_src_PART])/df[constants.Param_tgt_PART]).replace(np.NINF,0)
    df[constants.Param_diff_PUNCT]=((df[constants.Param_tgt_PUNCT]-df[constants.Param_src_PUNCT])/df[constants.Param_tgt_PUNCT])
    df[constants.Param_diff_countAbnormlWord]=(( df[constants.Param_tgt_countAbnormlWord]- df[constants.Param_src_countAbnormlWord])/ df[constants.Param_tgt_countAbnormlWord]).replace(np.Infinity,0)
    df[constants.Param_diff_ratioAbnormal]=((df[constants.Param_tgt_ratioAbnormal]-df[constants.Param_src_ratioAbnormal])/df[constants.Param_tgt_ratioAbnormal])
    df=df.fillna(0)
    df=df.replace(np.Infinity,0)
    return df
def PreProcess_MaskTime(df):
    
    print('Starting preprocessing ....')
    if df.isnull().sum().all().sum()>0:
        print('\n Data has some null values')
        print('\n Fill NaN value to 0')
        df=df.fillna(0)
        print('\n done')
        
    print('\n Create dummies with OneHot encoder for subjects and modalities')
    translator='subject_id'
    modality='modality'
    
    traslator_vectors = {}
    modality_vectors = {}
    i=1
    modality_vectors = pd.get_dummies(df[modality])
    traslator_vectors = pd.get_dummies(df[translator])

    for subject in constants.subjects:
        df[subject]=traslator_vectors[subject]

    for modality in constants.modalities:
        df[modality]=modality_vectors[modality]

    print('done \n Replace nan string with empty in aligned_edit and mt_text')
    #replace nan string in aligned_edit and mt_text
    df[constants.Param_aligned_edit].replace('nan','',inplace=True)
    df[constants.Param_mt_text].replace('nan','',inplace=True)
    
    print('done \n Creating new columns number of words and number of uniaue words for src_text, mt_text and tgt_text')
    # Number of words in src_text,mt_text,tgt_text
    df[constants.Param_num_word_src] = df['src_text'].apply(lambda x: len(str(x).split()))
    df[constants.Param_num_word_mt] = df[constants.Param_mt_text].apply(lambda x: len(str(x).split()))
    df[constants.Param_num_word_tgt] = df[constants.Param_tgt_text].apply(lambda x: len(str(x).split()))

    

    # Number of unigue words in src_text,mt_text,tgt_text
    #df["num_unique_src"] = df["src_text"].apply(lambda x: len(set(str(x).split())))
    df[constants.Param_num_word_unique_mt] = df[constants.Param_mt_text].apply(lambda x: len(set(str(x).split())))
    df[constants.Param_num_word_unique_tgt] = df[constants.Param_tgt_text].apply(lambda x: len(set(str(x).split())))

    
    # Number of unigue words in mt_text group by subject 
    constants.s_mt_t_wise=df.groupby(constants.Param_subject_id)[constants.Param_mt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_mt_t_wise = constants.s_mt_t_wise.apply(lambda x: len(set(str(x).split())))

    # Number of unigue words in tgt_text group by subject 
    constants.s_tgt_t_wise=df.groupby(constants.Param_subject_id)[constants.Param_tgt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_tgt_t_wise = constants.s_tgt_t_wise.apply(lambda x: len(set(str(x).split())))

    # Number of unigue words in mt_text group by modality 
    constants.s_mt_m_wise=df.groupby(constants.Param_modality)[constants.Param_mt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_mt_m_wise = constants.s_mt_m_wise.apply(lambda x: len(set(str(x).split())))
    
    # Number of unigue words in tgt_text group by modality
    constants.s_tgt_m_wise=df.groupby(constants.Param_modality)[constants.Param_tgt_text].agg(lambda x: ' '.join(set(x)))
    constants.s_tgt_m_wise= constants.s_tgt_m_wise.apply(lambda x: len(set(str(x).split())))

    # Calculating Ratio 300 & 1000
    #df[constants.Param_n300_ratio]=(df[constants.Param_n_pause_geq_300]/df[constants.Param_edit_time])*100
    #df[constants.Param_n1000_ratio]=(df[constants.Param_n_pause_geq_1000]/df[constants.Param_edit_time])*100

    # Setting unique numbers(above values) in the dataframe
    for subject in constants.subjects:
        df.loc[(df[constants.Param_subject_id] == subject), 'num_unique_mt_t_wise']=constants.s_mt_t_wise[subject]
        df.loc[(df[constants.Param_subject_id] == subject), 'num_unique_tgt_t_wise']=constants.s_tgt_t_wise[subject]

    for modality in constants.modalities:
        df.loc[(df[constants.Param_modality] == modality), 'num_unique_mt_m_wise']=constants.s_mt_m_wise[modality]
        df.loc[(df[constants.Param_modality] == modality), 'num_unique_tgt_m_wise']=constants.s_tgt_m_wise[modality]
    
    # Aligned text extraction
    new= df[constants.Param_aligned_edit].str.split("EVAL:", n = 1, expand = True)
    df[constants.Param_seperated_aligned_edit] =new[1]
    df[constants.Param_seperated_aligned_edit].replace('NONE','',inplace=True)
    df[constants.Param_num_of_I] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('I'))-1)
    df[constants.Param_num_of_D] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('D'))-1)
    df[constants.Param_num_of_S] = df[constants.Param_seperated_aligned_edit].apply(lambda x: len(str(x).split('S'))-1)
    df[constants.Param_num_of_IDS] = df[constants.Param_num_of_I]+df[constants.Param_num_of_D]+df[constants.Param_num_of_S]

    #new_df[[constants.Param_subject_id,constants.Param_aligned_edit,constants.Param_num_of_I,constants.Param_num_of_D,constants.Param_num_of_S,constants.Param_num_of_IDS]]
    
    df = df.drop(constants.Param_seperated_aligned_edit, 1)

    # linguistic src_text features + ratio Abnormal tokens
    text=constants.Param_src_text
    df[constants.Param_src_countWord3]=df[text].map( lambda x: countWords_Src(x,3))
    df[constants.Param_src_countWord4]=df[text].map( lambda x: countWords_Src(x,4))
    df[constants.Param_src_totalNumberOfTokens]=df[text].map( lambda x: getNumberOfTokens_Src(x))
    df[constants.Param_src_NOUN]=df[text].map( lambda x: getCount_Src(x,[NOUN]))
    df[constants.Param_src_VERB]=df[text].map( lambda x: getCount_Src(x,[VERB]))
    df[constants.Param_src_ADJ]=df[text].map( lambda x: getCount_Src(x,[ADJ]))
    df[constants.Param_src_ADV]=df[text].map( lambda x: getCount_Src(x,[ADV]))
    df[constants.Param_src_PROPN]=df[text].map( lambda x: getCount_Src(x,[PROPN]))
    df[constants.Param_src_PRON]=df[text].map( lambda x: getCount_Src(x,[PRON]))
    df[constants.Param_src_AUX]=df[text].map( lambda x: getCount_Src(x,[AUX]))
    df[constants.Param_src_CONJ]=df[text].map( lambda x: getCount_Src(x,[CONJ,CCONJ,SCONJ]))
    df[constants.Param_src_ADP]=df[text].map( lambda x: getCount_Src(x,[ADP]))
    df[constants.Param_src_DET]=df[text].map( lambda x: getCount_Src(x,[DET]))
    df[constants.Param_src_SYM]=df[text].map( lambda x: getCount_Src(x,[SYM]))
    df[constants.Param_src_PART]=df[text].map( lambda x: getCount_Src(x,[PART]))
    df[constants.Param_src_PUNCT]=df[text].map( lambda x: getCount_Src(x,[PUNCT]))
    df[constants.Param_src_countAbnormlWord]=(df[constants.Param_src_countWord4]+df[constants.Param_src_countWord3])-(df[constants.Param_src_DET]+df[constants.Param_src_PUNCT]+df[constants.Param_src_ADP])
    
    df[constants.Param_src_ratioAbnormal]=(df[constants.Param_src_countAbnormlWord]/df[constants.Param_src_totalNumberOfTokens])*100
    df[constants.Param_src_ratioAbnormal]=df[constants.Param_src_ratioAbnormal].replace(np.Infinity,0)
   
                                          


    # linguistic tgt_text features + ratio Abnormal tokens
    text=constants.Param_tgt_text
    df[constants.Param_tgt_countWord3]=df[text].map( lambda x: countWords(x,3))
    df[constants.Param_tgt_countWord4]=df[text].map( lambda x: countWords(x,4))
    df[constants.Param_tgt_totalNumberOfTokens]=df[text].map( lambda x: getNumberOfTokens(x))
    df[constants.Param_tgt_NOUN]=df[text].map( lambda x: getCount(x,[NOUN]))
    df[constants.Param_tgt_VERB]=df[text].map( lambda x: getCount(x,[VERB]))
    df[constants.Param_tgt_ADJ]=df[text].map( lambda x: getCount(x,[ADJ]))
    df[constants.Param_tgt_ADV]=df[text].map( lambda x: getCount(x,[ADV]))
    df[constants.Param_tgt_PROPN]=df[text].map( lambda x: getCount(x,[PROPN]))
    df[constants.Param_tgt_PRON]=df[text].map( lambda x: getCount(x,[PRON]))
    df[constants.Param_tgt_AUX]=df[text].map( lambda x: getCount(x,[AUX]))
    df[constants.Param_tgt_CONJ]=df[text].map( lambda x: getCount(x,[CONJ,CCONJ,SCONJ]))
    df[constants.Param_tgt_ADP]=df[text].map( lambda x: getCount(x,[ADP]))
    df[constants.Param_tgt_DET]=df[text].map( lambda x: getCount(x,[DET]))
    df[constants.Param_tgt_SYM]=df[text].map( lambda x: getCount(x,[SYM]))
    df[constants.Param_tgt_PART]=df[text].map( lambda x: getCount(x,[PART]))
    df[constants.Param_tgt_PUNCT]=df[text].map( lambda x: getCount(x,[PUNCT]))
    df[constants.Param_tgt_countAbnormlWord]=(df[constants.Param_tgt_countWord4]+df[constants.Param_tgt_countWord3])-(df[constants.Param_tgt_DET]+df[constants.Param_tgt_PUNCT]+df[constants.Param_tgt_ADP])
    df[constants.Param_tgt_ratioAbnormal]=(df[constants.Param_tgt_countAbnormlWord]/df[constants.Param_tgt_totalNumberOfTokens])*100
    df[constants.Param_tgt_ratioAbnormal]=df[constants.Param_tgt_ratioAbnormal]
    # linguistic featurs diff 
    df[constants.Param_diff_countWord3]=((df[constants.Param_tgt_countWord3]-df[constants.Param_src_countWord3])/df[constants.Param_tgt_countWord3])
    df[constants.Param_diff_countWord4]=((df[constants.Param_tgt_countWord4]-df[constants.Param_src_countWord4])/df[constants.Param_tgt_countWord4])
    df[constants.Param_diff_totalNumberOfTokens]=((df[constants.Param_tgt_totalNumberOfTokens]-df[constants.Param_src_totalNumberOfTokens])/df[constants.Param_tgt_totalNumberOfTokens])
    df[constants.Param_diff_NOUN]=((df[constants.Param_tgt_NOUN]-df[constants.Param_src_NOUN])/df[constants.Param_tgt_NOUN])
    df[constants.Param_diff_VERB]=((df[constants.Param_tgt_VERB]-df[constants.Param_src_VERB])/df[constants.Param_tgt_VERB])
    df[constants.Param_diff_ADJ]=((df[constants.Param_tgt_ADJ]-df[constants.Param_src_ADJ])/df[constants.Param_tgt_ADJ])
    df[constants.Param_diff_ADV]=((df[constants.Param_tgt_ADV]-df[constants.Param_src_ADV])/df[constants.Param_tgt_ADV])
    df[constants.Param_diff_PROPN]=((df[constants.Param_tgt_PROPN]-df[constants.Param_src_PROPN])/df[constants.Param_tgt_PROPN])
    df[constants.Param_diff_PRON]=((df[constants.Param_tgt_PRON]-df[constants.Param_src_PRON])/df[constants.Param_tgt_PRON])
    df[constants.Param_diff_AUX]=((df[constants.Param_tgt_AUX]-df[constants.Param_src_AUX])/df[constants.Param_tgt_AUX])
    df[constants.Param_diff_CONJ]=((df[constants.Param_tgt_CONJ]-df[constants.Param_src_CONJ])/df[constants.Param_tgt_CONJ])
    df[constants.Param_diff_ADP]=((df[constants.Param_tgt_ADP]-df[constants.Param_src_ADP])/df[constants.Param_tgt_ADP])
    df[constants.Param_diff_DET]=((df[constants.Param_tgt_DET]-df[constants.Param_src_DET])/df[constants.Param_tgt_DET])
    df[constants.Param_diff_SYM]=((df[constants.Param_tgt_SYM]-df[constants.Param_src_SYM])/df[constants.Param_tgt_SYM])
    df[constants.Param_diff_PART]=((df[constants.Param_tgt_PART]-df[constants.Param_src_PART])/df[constants.Param_tgt_PART]).replace(np.NINF,0)
    df[constants.Param_diff_PUNCT]=((df[constants.Param_tgt_PUNCT]-df[constants.Param_src_PUNCT])/df[constants.Param_tgt_PUNCT])
    df[constants.Param_diff_countAbnormlWord]=(( df[constants.Param_tgt_countAbnormlWord]- df[constants.Param_src_countAbnormlWord])/ df[constants.Param_tgt_countAbnormlWord]).replace(np.Infinity,0)
    df[constants.Param_diff_ratioAbnormal]=((df[constants.Param_tgt_ratioAbnormal]-df[constants.Param_src_ratioAbnormal])/df[constants.Param_tgt_ratioAbnormal])
    df=df.fillna(0)
    df=df.replace(np.Infinity,0)
    return df

def createMaindataFrame(odf):
    odf=PreProcess(odf)
    odf=odf.fillna(0)
    odf=odf.replace(np.Infinity,0)

    print('\n Creating main data frame ...')
    dfSubject=[]
    columns=[
        constants.Param_translator1,constants.Param_translator2,constants.Param_translator3,
        constants.Param_modality_ht,constants.Param_modality_pe1,constants.Param_modality_pe2,
        constants.Param_k_total,constants.Param_k_letter,constants.Param_k_copy,constants.Param_k_cut,constants.Param_k_digit,constants.Param_k_erase,
        constants.Param_k_nav,constants.Param_k_paste,constants.Param_k_symbol,constants.Param_k_white,
        constants.Param_n_delete,constants.Param_n_insert,constants.Param_n_shift,constants.Param_n_substitute,constants.Param_num_annotations,
        constants.Param_n_pause_geq_300,constants.Param_len_pause_geq_300,constants.Param_n_pause_geq_1000,constants.Param_len_pause_geq_1000,
        constants.Param_TER,constants.Param_BLEU,constants.Param_CHRF,
        constants.Param_n300_ratio,constants.Param_n1000_ratio,
        constants.Param_num_word_mt,constants.Param_num_word_tgt,constants.Param_num_words_per_edit_time,
        constants.Param_num_word_unique_mt,constants.Param_num_word_unique_tgt,
        constants.Param_num_unique_mt_t_wise,constants.Param_num_unique_tgt_t_wise,
        constants.Param_num_unique_mt_m_wise,constants.Param_num_unique_tgt_m_wise,
        constants.Param_num_of_I,constants.Param_num_of_D,constants.Param_num_of_S,constants.Param_num_of_IDS,
        
        constants.Param_src_NOUN,constants.Param_src_PROPN,constants.Param_src_VERB,constants.Param_src_ADJ,constants.Param_src_ADV,constants.Param_src_ADP,
        constants.Param_src_CONJ,constants.Param_src_PUNCT,constants.Param_src_DET,constants.Param_src_PRON,constants.Param_src_AUX,constants.Param_src_SYM,
        constants.Param_src_PART,
        constants.Param_src_countAbnormlWord,constants.Param_src_totalNumberOfTokens,constants.Param_src_ratioAbnormal,

        constants.Param_tgt_NOUN,constants.Param_tgt_PROPN,constants.Param_tgt_VERB,constants.Param_tgt_ADJ,constants.Param_tgt_ADV,constants.Param_tgt_ADP,
        constants.Param_tgt_CONJ,constants.Param_tgt_PUNCT,constants.Param_tgt_DET,constants.Param_tgt_PRON,constants.Param_tgt_AUX,constants.Param_tgt_SYM,
        constants.Param_tgt_PART,
        constants.Param_tgt_countAbnormlWord,constants.Param_tgt_totalNumberOfTokens,constants.Param_tgt_ratioAbnormal,

        constants.Param_diff_NOUN,constants.Param_diff_PROPN,constants.Param_diff_VERB,constants.Param_diff_ADJ,constants.Param_diff_ADV,constants.Param_diff_ADP,
        constants.Param_diff_CONJ,constants.Param_diff_PUNCT,constants.Param_diff_DET,constants.Param_diff_PRON,constants.Param_diff_AUX,constants.Param_diff_SYM,
        constants.Param_diff_PART,
        constants.Param_diff_countAbnormlWord,constants.Param_diff_totalNumberOfTokens,constants.Param_diff_ratioAbnormal


        ]
    dfSubject=odf[columns].copy()
    return dfSubject
def createMaskSubjectDataFrame(odf):
    odf=PreProcess_MaskSubject(odf)
    odf=odf.fillna(0)
    print('\n Creating mask subject data frame ...')
    dfSubject=[]
    columns=[
        constants.Param_modality_ht,constants.Param_modality_pe1,constants.Param_modality_pe2,
        constants.Param_edit_time,
        constants.Param_k_total,constants.Param_k_letter,constants.Param_k_copy,constants.Param_k_cut,constants.Param_k_digit,constants.Param_k_erase,
        constants.Param_k_nav,constants.Param_k_paste,constants.Param_k_symbol,constants.Param_k_white,
        constants.Param_n_delete,constants.Param_n_insert,constants.Param_n_shift,constants.Param_n_substitute,constants.Param_num_annotations,
        constants.Param_n_pause_geq_300,constants.Param_len_pause_geq_300,constants.Param_n_pause_geq_1000,
        constants.Param_len_pause_geq_1000,constants.Param_TER,constants.Param_BLEU,constants.Param_CHRF,
        constants.Param_n300_ratio,constants.Param_n1000_ratio,
        constants.Param_num_word_mt,constants.Param_num_word_tgt,constants.Param_num_words_per_edit_time,
        constants.Param_num_word_unique_mt,constants.Param_num_word_unique_tgt,
        constants.Param_num_unique_mt_m_wise,constants.Param_num_unique_tgt_m_wise,
        constants.Param_num_of_I,constants.Param_num_of_D,constants.Param_num_of_S,constants.Param_num_of_IDS,
        constants.Param_src_NOUN,constants.Param_src_PROPN,constants.Param_src_VERB,constants.Param_src_ADJ,constants.Param_src_ADV,constants.Param_src_ADP,
        constants.Param_src_CONJ,constants.Param_src_PUNCT,constants.Param_src_DET,constants.Param_src_PRON,constants.Param_src_AUX,constants.Param_src_SYM,
        constants.Param_src_PART,
        constants.Param_src_countAbnormlWord,constants.Param_src_totalNumberOfTokens,constants.Param_src_ratioAbnormal,

        constants.Param_tgt_NOUN,constants.Param_tgt_PROPN,constants.Param_tgt_VERB,constants.Param_tgt_ADJ,constants.Param_tgt_ADV,constants.Param_tgt_ADP,
        constants.Param_tgt_CONJ,constants.Param_tgt_PUNCT,constants.Param_tgt_DET,constants.Param_tgt_PRON,constants.Param_tgt_AUX,constants.Param_tgt_SYM,
        constants.Param_tgt_PART,
        constants.Param_tgt_countAbnormlWord,constants.Param_tgt_totalNumberOfTokens,constants.Param_tgt_ratioAbnormal,

        constants.Param_diff_NOUN,constants.Param_diff_PROPN,constants.Param_diff_VERB,constants.Param_diff_ADJ,constants.Param_diff_ADV,constants.Param_diff_ADP,
        constants.Param_diff_CONJ,constants.Param_diff_PUNCT,constants.Param_diff_DET,constants.Param_diff_PRON,constants.Param_diff_AUX,constants.Param_diff_SYM,
        constants.Param_diff_PART,
        constants.Param_diff_countAbnormlWord,constants.Param_diff_totalNumberOfTokens,constants.Param_diff_ratioAbnormal
        ]
    dfSubject=odf[columns].copy()
    return dfSubject
def createMaskModalityDataFrame(odf):
    odf=PreProcess_MaskModality(odf)
    odf=odf.fillna(0)
    print('\n Creating mask modality data frame ...')
    odf.isnull().sum()
    print('done \n data frame has been created!')
    
    columns=[
        constants.Param_translator1,constants.Param_translator2,constants.Param_translator3,
        constants.Param_edit_time,
        constants.Param_k_total,constants.Param_k_letter,constants.Param_k_copy,constants.Param_k_cut,constants.Param_k_digit,constants.Param_k_erase,
        constants.Param_k_nav,constants.Param_k_paste,constants.Param_k_symbol,constants.Param_k_white,
        #constants.Param_n_delete,constants.Param_n_insert,constants.Param_n_shift,constants.Param_n_substitute,
        constants.Param_num_annotations,
        constants.Param_n_pause_geq_300,constants.Param_len_pause_geq_300,constants.Param_n_pause_geq_1000,constants.Param_len_pause_geq_1000,
        #constants.Param_TER,constants.Param_BLEU,constants.Param_CHRF,
        constants.Param_n300_ratio,constants.Param_n1000_ratio,
        #constants.Param_num_word_mt,
        constants.Param_num_word_tgt,constants.Param_num_words_per_edit_time,
        #constants.Param_num_word_unique_mt,
        constants.Param_num_word_unique_tgt,
        #constants.Param_num_unique_mt_t_wise,
        constants.Param_num_unique_tgt_t_wise,
        #,constants.Param_num_of_I,constants.Param_num_of_D,constants.Param_num_of_S,constants.Param_num_of_IDS
        constants.Param_src_NOUN,constants.Param_src_PROPN,constants.Param_src_VERB,constants.Param_src_ADJ,constants.Param_src_ADV,constants.Param_src_ADP,
        constants.Param_src_CONJ,constants.Param_src_PUNCT,constants.Param_src_DET,constants.Param_src_PRON,constants.Param_src_AUX,constants.Param_src_SYM,
        constants.Param_src_PART,
        constants.Param_src_countAbnormlWord,constants.Param_src_totalNumberOfTokens,constants.Param_src_ratioAbnormal,

        constants.Param_tgt_NOUN,constants.Param_tgt_PROPN,constants.Param_tgt_VERB,constants.Param_tgt_ADJ,constants.Param_tgt_ADV,constants.Param_tgt_ADP,
        constants.Param_tgt_CONJ,constants.Param_tgt_PUNCT,constants.Param_tgt_DET,constants.Param_tgt_PRON,constants.Param_tgt_AUX,constants.Param_tgt_SYM,
        constants.Param_tgt_PART,
        constants.Param_tgt_countAbnormlWord,constants.Param_tgt_totalNumberOfTokens,constants.Param_tgt_ratioAbnormal,

        constants.Param_diff_NOUN,constants.Param_diff_PROPN,constants.Param_diff_VERB,constants.Param_diff_ADJ,constants.Param_diff_ADV,constants.Param_diff_ADP,
        constants.Param_diff_CONJ,constants.Param_diff_PUNCT,constants.Param_diff_DET,constants.Param_diff_PRON,constants.Param_diff_AUX,constants.Param_diff_SYM,
        constants.Param_diff_PART,
        constants.Param_diff_countAbnormlWord,constants.Param_diff_totalNumberOfTokens,constants.Param_diff_ratioAbnormal
        ]
    dfModalities=odf[columns].copy()
    return dfModalities   
def createMaskTimeDataFrame(odf):
    odf=PreProcess_MaskTime(odf)
    odf=odf.fillna(0)
    print('\n Creating mask time data frame ...')

    dfTime=[]
    columns=[
        constants.Param_translator1,constants.Param_translator2,constants.Param_translator3,
        constants.Param_modality_ht,constants.Param_modality_pe1,constants.Param_modality_pe2,
        constants.Param_k_total,constants.Param_k_letter,constants.Param_k_copy,constants.Param_k_cut,constants.Param_k_digit,constants.Param_k_erase,
        constants.Param_k_nav,constants.Param_k_paste,constants.Param_k_symbol,constants.Param_k_white,
        constants.Param_n_delete,constants.Param_n_insert,constants.Param_n_shift,constants.Param_n_substitute,constants.Param_num_annotations,
        #constants.Param_n_pause_geq_300,constants.Param_len_pause_geq_300,constants.Param_n_pause_geq_1000,constants.Param_len_pause_geq_1000,
        constants.Param_TER,constants.Param_BLEU,constants.Param_CHRF,
        #constants.Param_n300_ratio,constants.Param_n1000_ratio,
        constants.Param_num_word_mt,constants.Param_num_word_tgt,
        constants.Param_num_word_unique_mt,constants.Param_num_word_unique_tgt,
        constants.Param_num_unique_mt_t_wise,constants.Param_num_unique_tgt_t_wise,
        constants.Param_num_unique_mt_m_wise,constants.Param_num_unique_tgt_m_wise,
        constants.Param_num_of_I,constants.Param_num_of_D,constants.Param_num_of_S,constants.Param_num_of_IDS,
        constants.Param_src_NOUN,constants.Param_src_PROPN,constants.Param_src_VERB,constants.Param_src_ADJ,constants.Param_src_ADV,constants.Param_src_ADP,
        constants.Param_src_CONJ,constants.Param_src_PUNCT,constants.Param_src_DET,constants.Param_src_PRON,constants.Param_src_AUX,constants.Param_src_SYM,
        constants.Param_src_PART,
        constants.Param_src_countAbnormlWord,constants.Param_src_totalNumberOfTokens,constants.Param_src_ratioAbnormal,

        constants.Param_tgt_NOUN,constants.Param_tgt_PROPN,constants.Param_tgt_VERB,constants.Param_tgt_ADJ,constants.Param_tgt_ADV,constants.Param_tgt_ADP,
        constants.Param_tgt_CONJ,constants.Param_tgt_PUNCT,constants.Param_tgt_DET,constants.Param_tgt_PRON,constants.Param_tgt_AUX,constants.Param_tgt_SYM,
        constants.Param_tgt_PART,
        constants.Param_tgt_countAbnormlWord,constants.Param_tgt_totalNumberOfTokens,constants.Param_tgt_ratioAbnormal,

        constants.Param_diff_NOUN,constants.Param_diff_PROPN,constants.Param_diff_VERB,constants.Param_diff_ADJ,constants.Param_diff_ADV,constants.Param_diff_ADP,
        constants.Param_diff_CONJ,constants.Param_diff_PUNCT,constants.Param_diff_DET,constants.Param_diff_PRON,constants.Param_diff_AUX,constants.Param_diff_SYM,
        constants.Param_diff_PART,
        constants.Param_diff_countAbnormlWord,constants.Param_diff_totalNumberOfTokens,constants.Param_diff_ratioAbnormal
        ]
    dfTime=odf[columns].copy()
    return dfTime


In [10]:
new_dfTime=createMaskTimeDataFrame(new_df)

Starting preprocessing ....

 Create dummies with OneHot encoder for subjects and modalities
done 
 Replace nan string with empty in aligned_edit and mt_text
done 
 Creating new columns number of words and number of uniaue words for src_text, mt_text and tgt_text

 Creating mask time data frame ...


In [11]:
new_dfTime = new_dfTime.replace(np.NINF,0)

In [12]:
new_dfTime.iloc[0]

t1                          1.000000
t2                          0.000000
t3                          0.000000
ht                          1.000000
pe1                         0.000000
                              ...   
diff_SYM                    0.000000
diff_PART                   0.000000
diff_countAbnormlWord      -1.000000
diff_totalNumberOfTokens    0.256410
diff_ratioAbnormal         -1.689655
Name: 0, Length: 84, dtype: float64

In [13]:
label_edit_time = new_df["edit_time"]
label_edit_time

0       149.979996
1       128.078995
2       141.500000
3       151.317993
4        66.817001
           ...    
1165     61.193001
1166    143.360001
1167    154.690002
1168    182.785995
1169    286.171997
Name: edit_time, Length: 1170, dtype: float64

 #### Splitting the data set into 80% training and  20%test

In [14]:
x_train_et,x_test_et,y_train_et,y_test_et = train_test_split(new_dfTime, label_edit_time, test_size = 0.2, random_state = 90)

In [15]:
x_train_et.shape, x_test_et.shape,y_train_et.shape,y_test_et.shape

((936, 84), (234, 84), (936,), (234,))

#### Training Random Forest regressor with hypertuned pararmeters 

In [16]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [i for i in range(4,20)],
    'min_samples_leaf': [2,3, 4, 5],
    'min_samples_split': [i for i in (range(2,8))],
    'n_estimators': [i for i in range(10,40)]
}

In [17]:
RF_et_tuning = RandomForestRegressor()

#### Using K-Fold using 10 splits for hyper-tuning

In [18]:
cv = KFold(n_splits=10,random_state=None,shuffle=False)

In [19]:
grid_search_et = GridSearchCV(estimator = RF_et_tuning, param_grid = param_grid, 
                          cv = cv)

In [20]:
grid_search_et.fit(x_train_et, y_train_et)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True],
                         'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                       16, 17, 18, 19],
                         'min_samples_leaf': [2, 3, 4, 5],
                         'min_samples_split': [2, 3, 4, 5, 6, 7],
                         'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18,
                                          19, 20, 21, 22, 23, 24, 25, 26, 27,
                                          28, 29, 30, 31, 32, 33, 34, 35, 36,
                                          37, 38, 39]})

In [21]:
print(grid_search_et.best_params_)

{'bootstrap': True, 'max_depth': 9, 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 20}


In [22]:
print(grid_search_et.best_estimator_)

RandomForestRegressor(max_depth=9, min_samples_leaf=3, min_samples_split=7,
                      n_estimators=20)


In [23]:
y_pred_et_tuning = grid_search_et.predict(x_test_et)
y_pred_et_tuning.shape

(234,)

#### Evaluation of the model using root mean square error

In [24]:
rmse_et = (np.sqrt(mean_squared_error(y_test_et,y_pred_et_tuning)))
print("RMSE",rmse_et)

RMSE 80.03425347327406


### Feature importance of the edit time prediction model

In [25]:
# Get numerical feature importances
importances = list(grid_search_et.best_estimator_.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(new_dfTime.columns, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: k_total              Importance: 0.47
Variable: k_cut                Importance: 0.04
Variable: k_nav                Importance: 0.03
Variable: tgt_ratioAbnormal    Importance: 0.03
Variable: k_letter             Importance: 0.02
Variable: k_symbol             Importance: 0.02
Variable: num_unique_tgt_t_wise Importance: 0.02
Variable: diff_ADJ             Importance: 0.02
Variable: t2                   Importance: 0.01
Variable: k_copy               Importance: 0.01
Variable: k_erase              Importance: 0.01
Variable: k_white              Importance: 0.01
Variable: n_substitute         Importance: 0.01
Variable: bleu                 Importance: 0.01
Variable: chrf                 Importance: 0.01
Variable: num_word_tgt         Importance: 0.01
Variable: num_word_unique_tgt  Importance: 0.01
Variable: num_of_S             Importance: 0.01
Variable: num_of_IDS           Importance: 0.01
Variable: src_NOUN             Importance: 0.01
Variable: src_ADJ              Importan

## Evaluating the model with mask edit time test dataset

In [26]:
dataDir='dataset/IK_NLP_22_PESTYLE'
dataset_test_time = load_dataset("GroNLP/ik-nlp-22_pestyle", "mask_time", data_dir=dataDir)

Using custom data configuration mask_time-c8d896e19c9bb48e
Reusing dataset ik_nlp22_pe_style (C:\Users\amit\.cache\huggingface\datasets\GroNLP___ik_nlp22_pe_style\mask_time-c8d896e19c9bb48e\1.0.0\3bbf0fda4806257149c2beb42c6cd20db6f79dac9ae2498f44be55fa7a953d51)


  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
dataset_test_modality = load_dataset("GroNLP/ik-nlp-22_pestyle", "mask_modality", data_dir=dataDir)

Using custom data configuration mask_modality-c8d896e19c9bb48e
Reusing dataset ik_nlp22_pe_style (C:\Users\amit\.cache\huggingface\datasets\GroNLP___ik_nlp22_pe_style\mask_modality-c8d896e19c9bb48e\1.0.0\3bbf0fda4806257149c2beb42c6cd20db6f79dac9ae2498f44be55fa7a953d51)


  0%|          | 0/1 [00:00<?, ?it/s]

#### Mask edit time test dataset pre-processing

In [28]:
test_time = dataset_test_time["test"]
test_time_df = pd.DataFrame(data=test_time)
test_time_pre_df=createMaskTimeDataFrame(test_time_df)
test_time_pre_df = test_time_pre_df.replace(np.NINF,0)
test_time_pre_df.shape

Starting preprocessing ....

 Create dummies with OneHot encoder for subjects and modalities
done 
 Replace nan string with empty in aligned_edit and mt_text
done 
 Creating new columns number of words and number of uniaue words for src_text, mt_text and tgt_text

 Creating mask time data frame ...


(120, 84)

#### Prediction from mask edit time test dataset

In [29]:
ytest_pred_et = grid_search_et.predict(test_time_pre_df)
ytest_pred_et.shape

(120,)

#### Obtaining the true edit time from masked modality test dataset

In [30]:
test_modality = dataset_test_modality["test"]
test_modality_df = pd.DataFrame(data=test_modality)
test_modality_df.shape

(120, 20)

In [31]:
test_edit_times=test_modality_df["edit_time"]

#### Evaluation of the model on mask edit time test dataset

In [32]:
rmse_et = (np.sqrt(mean_squared_error(test_edit_times,ytest_pred_et)))
print("RMSE",rmse_et)

RMSE 59.08352459614063
